In [4]:
from __future__ import annotations

import glob
import os
import re
import time

import supersuit as ss
from stable_baselines3 import PPO, DQN
from stable_baselines3.ppo import MlpPolicy

from pettingzoo.sisl import multiwalker_v9
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback,CheckpointCallback
from stable_baselines3.common.results_plotter import load_results


logdir = "logs"


def train_butterfly_supersuit(
    env_fn, steps: int = 10_000, seed: int | None = 0, **env_kwargs
):
    # Train a single model to play as each agent in a cooperative Parallel environment
    env = env_fn.parallel_env(**env_kwargs, n_walkers = 2)
    #env = Monitor(env)

    env.reset(seed=seed)

    print(f"Starting training on {str(env.metadata['name'])}.")
    
    env = ss.frame_stack_v1(env, 3)
    env = ss.pettingzoo_env_to_vec_env_v1(env)
    env = ss.concat_vec_envs_v1(env, 8, num_cpus=2, base_class="stable_baselines3")

    #checlpoint callback
    checkpoint_callback = CheckpointCallback(save_freq=1000, save_path='./chekpoint_models/',
                                         name_prefix='rl_model')

    model = PPO(
        MlpPolicy,
        env,
        verbose=2,
        learning_rate=3e5,
        batch_size=1024,
        normalize_advantage=True,
        n_steps=4096,
        n_epochs=20,
        gae_lambda=0.95,
        gamma=0.99,
        clip_range=0.3,
        ent_coef=0.001,
        tensorboard_log=logdir,
    )
    

    model.learn(total_timesteps=100_000, callback=checkpoint_callback)

    model.save(f"{env.unwrapped.metadata.get('name')}_{time.strftime('%Y%m%d-%H%M%S')}")

    print("Model has been saved.")

    print(f"Finished training on {str(env.unwrapped.metadata['name'])}.")

    env.close()


def eval(env_fn, num_games: int = 100, render_mode: str | None = None, **env_kwargs):
    # Evaluate a trained agent vs a random agent
    env = env_fn.env(render_mode=render_mode, **env_kwargs, n_walkers = 2)
    
    # Apply the same frame stacking to the evaluation environment
    env = ss.frame_stack_v1(env, 3)
    

    print(
        f"\nStarting evaluation on {str(env.metadata['name'])} (num_games={num_games}, render_mode={render_mode})"
    )

    try:
        latest_policy = max(
            glob.glob(f"{env.metadata['name']}*.zip"), key=os.path.getctime
        )
    except ValueError:
        print("Policy not found.")
        exit(0)

    model = PPO.load(latest_policy)

    rewards = {agent: 0 for agent in env.possible_agents}

    # Note: We train using the Parallel API but evaluate using the AEC API
    # SB3 models are designed for single-agent settings, we get around this by using the same model for every agent
    for i in range(num_games):
        env.reset(seed=i)   

        for agent in env.agent_iter():
            
            obs, reward, termination, truncation, info = env.last()

            for a in env.agents:
                rewards[a] += env.rewards[a]
            if termination or truncation:
                break
            else:
                act = model.predict(obs, deterministic=True)[0]

            env.step(act)
    env.close()

    avg_reward = sum(rewards.values()) / len(rewards.values())
    print("Rewards: ", rewards)
    print(f"Avg reward: {avg_reward}")
    return avg_reward

In [5]:
env_fn = multiwalker_v9
env_kwargs = {}

In [6]:
train_butterfly_supersuit(env_fn, **env_kwargs)

Starting training on multiwalker_v9.


Using cuda device
Logging to logs/PPO_18
------------------------------
| time/              |       |
|    fps             | 2956  |
|    iterations      | 1     |
|    time_elapsed    | 22    |
|    total_timesteps | 65536 |
------------------------------


ValueError: Expected parameter scale (Tensor of shape (1024, 4)) of distribution Normal(loc: torch.Size([1024, 4]), scale: torch.Size([1024, 4])) to satisfy the constraint GreaterThan(lower_bound=0.0), but found invalid values:
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], device='cuda:0', grad_fn=<MulBackward0>)

In [ ]:

eval(env_fn, num_games=2, render_mode="human", **env_kwargs)